In [1]:
import os
import os.path as osp
import collections as C
import json

In [2]:
benchmarks = ['formal_math500', 'minif2f_solving', 'putnam_solving']
benchmark_lengths = [387, 375, 324]
metrics = ['solved', 'proven', 'ne_submitted']

results_root = './baseline_results/'

results_all = C.defaultdict(    # Paradigm
    lambda : C.defaultdict(              # Model
        dict                    # Benchmark
    )
)

In [3]:
for paradigm in os.listdir(results_root):
    for model in os.listdir(osp.join(results_root, paradigm)):
        exp_dir = osp.join(results_root, paradigm, model)
        for (benchmark, benchmark_l) in zip(benchmarks, benchmark_lengths):
            result = dict()
            with open(osp.join(exp_dir, f'solving.{benchmark}.jsonl'), 'r') as f:
                solving_results = [json.loads(l) for l in f.readlines()]
            n_solved = len([
                s for s in solving_results if len(s.get('submission', '')) > 0 and s['eq_proof'] is not None
            ])
            n_submitted = len([
                s for s in solving_results if len(s.get('submission', '')) > 0
            ])
            result['solved'] = n_solved / benchmark_l
            result['ne_submitted'] = (n_submitted- n_solved) / benchmark_l
            if osp.exists(osp.join(exp_dir, f'proving.{benchmark}.jsonl')):
                with open(osp.join(exp_dir, f'proving.{benchmark}.jsonl'), 'r') as f:
                    proving_results = [json.loads(l) for l in f.readlines()]
                    result['proven'] = len([
                        s for s in proving_results if s['search_result']['success']
                    ]) / benchmark_l
            results_all[paradigm][model][benchmark] = result

In [4]:
for k in ['benchmark', 'paradigm', 'model'] + metrics:
    print(k, end='\t')
print()

for benchmark in benchmarks:
    for paradigm, results_by_models in results_all.items():
        for model, results_by_benchmarks in results_by_models.items():
            result = results_by_benchmarks[benchmark]
            print(benchmark, paradigm, model, sep='\t', end='\t')
            for m in metrics:
                print(result.get(m, ''), end='\t')
            print()

benchmark	paradigm	model	solved	proven	ne_submitted	
formal_math500	whole_proof_generation	theoremllama	0.16020671834625322	0.04392764857881137	0.15503875968992248	
formal_math500	whole_proof_generation	deepseek-prover-v1.5	0.2222222222222222	0.46511627906976744	0.14470284237726097	
formal_math500	proof_search	leanstar	0.2351421188630491	0.43410852713178294	0.20930232558139536	
formal_math500	proof_search	internlm2.5-step-prover	0.23772609819121446	0.4754521963824289	0.1937984496124031	
formal_math500	prompting_methods	icl	0.13695090439276486		0.0	
formal_math500	prompting_methods	hybrid_cot	0.15503875968992248		0.0103359173126615	
minif2f_solving	whole_proof_generation	theoremllama	0.13066666666666665	0.07733333333333334	0.088	
minif2f_solving	whole_proof_generation	deepseek-prover-v1.5	0.224	0.536	0.10933333333333334	
minif2f_solving	proof_search	leanstar	0.24266666666666667	0.49333333333333335	0.144	
minif2f_solving	proof_search	internlm2.5-step-prover	0.27466666666666667	0.50666666